<a href="https://colab.research.google.com/github/seein2/capstone/blob/seein/python/model_practice/KcELECTRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install datasets
!pip install pandas
!pip install torch
!pip install transformers
!pip install openpyxl
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from transformers import AdamW, get_scheduler
from tqdm import tqdm

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# KcELECTRA 모델과 토크나이저 불러오기
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = pd.read_excel('./drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')
data

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 특수문자 제거
def remove_special_characters(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', text)

data['Sentence'] = data['Sentence'].apply(remove_special_characters)

In [ ]:
data['Sentence']

In [ ]:
# 라벨링
data.loc[(data['Emotion'] == "공포"), 'Emotion'] = 0       #공포=> 0
data.loc[(data['Emotion'] == "놀람"), 'Emotion'] = 1       #놀람 => 1
data.loc[(data['Emotion'] == "분노"), 'Emotion'] = 2       #분노 => 2
data.loc[(data['Emotion'] == "슬픔"), 'Emotion'] = 3       #슬픔 => 3
data.loc[(data['Emotion'] == "중립"), 'Emotion'] = 4       #중립 => 4
data.loc[(data['Emotion'] == "행복"), 'Emotion'] = 5       #행복 => 5
data.loc[(data['Emotion'] == "혐오"), 'Emotion'] = 6       #혐오 => 6


data_list = []
for ques, label in zip(data['Sentence'], data['Emotion']):
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=2000)
print(len(dataset_train), len(dataset_test))

28945 9649


In [ ]:
# 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence, label = self.data[idx]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label = torch.tensor(int(label), dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

In [ ]:
# 학습 데이터셋과 테스트 데이터셋 준비
train_dataset = EmotionDataset(dataset_train, tokenizer)
test_dataset = EmotionDataset(dataset_test, tokenizer)

In [ ]:
# DataLoader 정의 (배치 단위로 데이터를 불러오기 위함)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
# 옵티마이저 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프 정의
epochs = 6

# 학습률 스케줄러 정의
num_training_steps = epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
for epoch in range(epochs):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델에 입력 데이터 전달 후 출력 획득
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # 역전파 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

100%|██████████| 453/453 [02:21<00:00,  3.20it/s]


Epoch 1, Loss: 1.2313180846094296


100%|██████████| 453/453 [02:21<00:00,  3.20it/s]


Epoch 2, Loss: 0.9055442046908616


100%|██████████| 453/453 [02:21<00:00,  3.20it/s]


...

Epoch 4, Loss: 0.4432815547876274


100%|██████████| 453/453 [02:21<00:00,  3.20it/s]


Epoch 5, Loss: 0.2847059770234373


100%|██████████| 453/453 [02:21<00:00,  3.20it/s]

Epoch 6, Loss: 0.1893336171033498


In [ ]:
model.eval()  # 평가 모드로 설정
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy}")

100%|██████████| 151/151 [00:17<00:00,  8.84it/s]

Test Accuracy: 0.5705254430510934


In [ ]:
def analyze_diary(model, tokenizer, diary_text, temperature=1.5):
    model.eval()
    encoding = tokenizer.encode_plus(
        diary_text,
        add_special_tokens=True,
        max_length=128,
        return_attention_mask=True,
        return_tensors='pt',
        padding='max_length',
        truncation=True
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits / temperature  # 온도 스케일링 적용
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    emotions = ["공포", "분노", "놀람", "슬픔", "중립", "행복", "혐오"]
    intensity = probabilities.squeeze().cpu().numpy()

    results = {emotion: round(intensity[i], 2) for i, emotion in enumerate(emotions)}
    return results

In [ ]:
# 사용자가 입력한 일기 예시
diary_text = "오늘 되게 피곤한데 아침부터 나와서 일하니까 너무 힘들다 너무 좆같아. 하지만 일 끝나고 집가서 야식 먹으면서 넷플릭스 볼 생각에 설레"
analyze_diary(model, tokenizer, diary_text)

{'공포': 0.01,
 '분노': 0.01,
 '놀람': 0.08,
 '슬픔': 0.72,
 '중립': 0.02,
 '행복': 0.12,
 '혐오': 0.05}